In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import mstats
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
mapping_file = pd.read_csv('../Data/mapping_file.csv', index_col=0)
print(mapping_file.shape)
mapping_file.head()

(143, 28)


,genotype,fertility,amf,rep,group,shoot_mass,root_mass,total_mass,ratio_root_to_shoot,root_colonization,...,Fe,Zn,Cu,Mn,Length,AvgDiam,Forks,SurfArea,Origin,Percent
#SampleID,,,,,,,,,,,,,,,,,,,,,
L1,1,P,Y,1,Common (C. dactylon),29.961,4.46,34.421,0.129572,43,...,205.131,40.296,24.655,134.694,162.8464,0.2905,904.0,14.8643,African,100%
L10,1,PPP,Y,2,Common (C. dactylon),26.283,10.16,36.443,0.278792,36,...,291.100,29.000,15.800,66.900,329.7942,0.2978,2620.0,30.8595,African,100%
L100,9,PP,Y,5,Common (C. dactylon),18.142,6.82,24.962,0.273215,21,...,48.811,38.705,9.024,55.564,229.4975,0.3495,1577.0,25.1969,Asian,100%
L101,9,PP,Y,1,Common (C. dactylon),25.649,10.88,36.529,0.297846,26,...,290.800,26.900,16.600,70.900,216.1839,0.3708,1395.0,25.1835,Asian,100%
L102,9,PP,Y,2,Common (C. dactylon),25.664,12.66,38.324,0.330341,39,...,265.700,29.200,20.700,83.900,302.0509,0.2743,1827.0,26.0297,Asian,100%


In [4]:
mapping_file[['Origin']].value_counts()

Origin 
Asian      59
African    48
Unknown    36
dtype: int64

In [8]:
# ANOVA

p_values_1 = []
p_values_2 = []
p_values_3 = []
p_values_4 = []
f_stats_1 = []
f_stats_2 = []
f_stats_3 = []
f_stats_4 = []
columns = ['shoot_mass', 'root_mass', 'total_mass', 'ratio_root_to_shoot',
       'root_colonization', 'TN',
       'P', 'Ca', 'K', 'Mg', 'Na', 'S', 'Fe', 'Zn', 'Cu', 'Mn', 'Length',
       'AvgDiam', 'Forks', 'SurfArea']
for col in columns:
#     print(col)
    model = ols(col + ' ~ C(genotype) + C(fertility) + C(genotype):C(fertility)', data=mapping_file).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    f_stats_1.append(anova_table['F'][0])
    f_stats_2.append(anova_table['F'][1])
    f_stats_3.append(anova_table['F'][2])
    p_values_1.append(anova_table['PR(>F)'][0])
    p_values_2.append(anova_table['PR(>F)'][1])
    p_values_3.append(anova_table['PR(>F)'][2])
    
    model = ols(col + ' ~ C(Origin)', data=mapping_file).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    f_stats_4.append(anova_table['F'][0])
    p_values_4.append(anova_table['PR(>F)'][0])
anova_df = pd.DataFrame({'Measurement': columns,'F stat (G)':f_stats_1,'P value (G)':p_values_1,
                         'F stat (F)':f_stats_2,'P value (F)':p_values_2,
                         'F stat (GxF)':f_stats_3,'P value (GxF)':p_values_3,
                        'F stat (Origin)':f_stats_4,'P value (Origin)':p_values_4})
anova_df = anova_df.replace({'shoot_mass': 'Shoot mass','root_mass':'Root mass', 'total_mass':'Total mass',
                 'ratio_root_to_shoot':'Root/shoot ratio','root_colonization':'Root colonization',
                  'inoculum':'Inoculum', 'AvgDiam':'Diameter (avg)', 'SurfArea':'Surface area',
                    'total_biomass_esponse':'Total biomass response', 'root_biomass_response':'Root biomass response',
                             'shoot_biomass_response':'Shoot biomass response'})
for col in ['F stat (G)','F stat (F)','F stat (GxF)','F stat (Origin)']:
    anova_df[col] = [round(x, 1) for x in anova_df[col]]
for col in ['P value (G)', 'P value (F)','P value (GxF)','P value (Origin)']:
    anova_df[col] = anova_df[col].astype('float')
    anova_df[col] = [str(round(x, 3)) if x > 0.001 else '< 0.001' for x in anova_df[col]]
anova_df.head()

,Measurement,F stat (G),P value (G),F stat (F),P value (F),F stat (GxF),P value (GxF),F stat (Origin),P value (Origin)
0,Shoot mass,10.9,< 0.001,15.2,< 0.001,1.9,0.015,9.3,< 0.001
1,Root mass,6.8,< 0.001,0.1,0.932,1.3,0.213,10.6,< 0.001
2,Total mass,11.1,< 0.001,3.6,0.031,1.4,0.126,15.0,< 0.001
3,Root/shoot ratio,4.0,< 0.001,3.5,0.034,1.4,0.131,4.1,0.018
4,Root colonization,1.7,0.082,0.1,0.892,0.8,0.758,1.3,0.264


In [9]:
anova_df.to_csv('../Output/genotypexfertility_anova_table_origin_KW.csv', index=False)